##### GPU Check
-----
Check if GPUs are being used, we have already installed tensroflow for GPU in our conda environment

In [ ]:
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))
print(len(tf.config.experimental.list_physical_devices('GPU')))

##### Import Libraries
---

In [37]:

import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
from sklearn.utils import resample
from sklearn.decomposition import PCA
from sklearn.preprocessing import MinMaxScaler
from sklearn.naive_bayes import MultinomialNB
import collections
import matplotlib.pyplot as plt
import pickle
import configparser
from yellowbrick.text import FreqDistVisualizer
import time
import random
%matplotlib inline

GA import

In [23]:
from deap import base, creator, tools, algorithms

##### Read config file
---

In [130]:
config_parser = configparser.ConfigParser()
config_parser.read_file(open('./Config.txt'))

##### Functions
---

In [4]:
def get_config_var(experiment_name, var):
    return config_parser.get(experiment_name, var)

In [5]:
def class_distribution():
    print('Counts of proteins:\n',corona_seq.groupby('class').size())
    print('\nPercentage of the distribution of labelled records:')
    print(round(corona_seq.groupby('class').size()/corona_seq.shape[0]*100,2))

In [6]:
def print_data_shape(train_s, test_s, ytrain_s, ytest_s, y_test):
    print('\n')
    print('Data shape - Train, Test:',train_s,',',test_s)
    print('Label shape - Train, Test:',ytrain_s,',',ytest_s)
    print('\n')
    counter = collections.Counter(y_test)
    counter.most_common()
    print('Test Label values and counts:\n',sorted(counter.items()))

In [7]:
def get_cv_train_test_data():
    X_train, X_test, y_train, y_test = train_test_split(corona_seq['joined_words'],
                                                        y_corona, 
                                                        shuffle = True,test_size = 0.25, 
                                                        random_state=0, stratify=y_corona)

    X_train = cv.fit_transform(X_train)
    X_test = cv.transform(X_test)

    X_train = get_array_from_sparsematrix(X_train)
    X_test = get_array_from_sparsematrix(X_test)

    train_s = X_train.shape
    test_s = X_test.shape
    ytrain_s = y_train.shape
    ytest_s = y_test.shape

    print_data_shape(train_s, test_s, ytrain_s, ytest_s, y_test)
    return X_train, X_test, y_train, y_test

In [117]:
def print_word_frequencies():
    plt.figure(figsize=(20,10))
    plt.rc('xtick', labelsize=20) 
    features = cv.get_feature_names()
    visualizer = FreqDistVisualizer(features=features, orient='v')
    visualizer.fit(X_train)   

In [9]:
def get_array_from_sparsematrix(sparsematrix):
    ret_array = sparsematrix.todense()
    return ret_array

In [10]:
def clfMultinomialNB(X_train, X_test, y_train, y_test):
    start = time.time()
    print ('\n\nAlgorithm Name: MultinomialNB')
    print('___________________________________________________________________\n')
    print('\ntraining model.........')
    clf = MultinomialNB(alpha=0.1)
    clf.fit(X_train, y_train)
    print('\ntesting.........\n')
    y_pred = predict(clf,X_test)
    print_metrices(y_test, y_pred)
    return clf

In [11]:
def predict(classifier, testdata, dense =False):
    if dense:
        testdata = testdata.todense()
    return classifier.predict(testdata)

In [12]:
def dimension_reduction_pca(X_train, X_test,no_of_components=9):
    pca = PCA(n_components=9)
    
    X_pca_train = pca.fit_transform(X_train)
    X_pca_test = pca.transform(X_test)

    # Output PCA variance results
    #print("Singular values = \n",pca.singular_values_)  # Not required to print
    print("\nProportion of variance = \n",pca.explained_variance_ratio_) # Required to print
    print("\nTotal of variance covered  = \n",round(sum(pca.explained_variance_ratio_),2)*100, "%")

    scaler, X_scaled_train = minmaxscaler_fit_transform(X_pca_train)
    X_scaled_test = minmaxscaler_transform(scaler, X_pca_test)
    return X_scaled_train, X_scaled_test

In [13]:
def minmaxscaler_fit_transform(array_object):
  scaler = MinMaxScaler()
  return scaler, scaler.fit_transform(array_object)

In [14]:
def minmaxscaler_transform(scaler, array_object):
  return scaler.transform(array_object)

In [15]:
def get_metrics(y_test, y_predicted):
    accuracy = accuracy_score(y_test, y_predicted)
    precision = precision_score(y_test, y_predicted, average='weighted', zero_division=0.0)
    recall = recall_score(y_test, y_predicted, average='weighted', zero_division=0.0)
    f1 = f1_score(y_test, y_predicted, average='weighted')
    return accuracy, precision, recall, f1

In [16]:
def print_header():
    print('___________________________________________________________________\n')
    experiment_info = get_config_var('Jupyter Experiment', 'name')
    min_ngram = get_config_var('Jupyter Experiment', 'min_ngram')
    max_ngram = get_config_var('Jupyter Experiment', 'max_ngram')
    kmersize =get_config_var('Jupyter Experiment', 'kmersize')
    dim_reduction_type = get_config_var('Jupyter Experiment', 'dim_reduction_type')
    
    print('Experiment Information:', experiment_info,' with ',dim_reduction_type,'\n')
    print('___________________________________________________________________\n')
    print('Pre-processing - kmer size:', kmersize)
    print('Pre-processing - min_ngram and max_ngram:',min_ngram,',',max_ngram)
    print('dim_reduction_type:',dim_reduction_type)    

In [17]:
def print_confusionmatrix(y_test, y_pred):
    print("\nConfusion matrix:\n")
    print(pd.crosstab(pd.Series(y_test, name='Actual'), pd.Series(y_pred, name='Predicted')))

In [18]:
def print_accuracy_precision_recall(y_test, y_pred):
    accuracy, precision, recall, f1 = get_metrics(y_test, y_pred)
    print("accuracy\t= %.6f \nprecision\t= %.6f \nrecall\t\t= %.6f \nf1\t\t= %.6f" % (accuracy, precision, recall, f1))  
    #return accuracy, precision, recall, f1

In [19]:
def print_metrices(y_test, y_pred):
    print_header()
    print('\n___________________________________________________________________')
    print("Results as below:") 
    print('___________________________________________________________________\n')
    print_confusionmatrix(y_test, y_pred)
    print("\nMatrices:\n")
    print_accuracy_precision_recall(y_test, y_pred)
    end = time.time()
    print('___________________________________________________________________')
    print('\nElapsed Time:', (end - start)/60)
    print('\n')

##### Prepare data
---------------------

restore pickle object

In [104]:
with open('./source/corona_seq.pkl','rb') as pickle_file:
  corona_seq = pickle.load(pickle_file)

Shuffle

In [21]:
corona_seq = corona_seq.sample(frac=1,random_state=4).reset_index(drop=True)

In [22]:
class_distribution()

Counts of proteins:
 class
envelope protein                509
membrane glycoprotein           413
nucleocapsid phosphoprotein    1108
orf10 protein                    83
orf3 protein                    711
orf6 protein                    178
orf7a protein                   406
orf7b protein                   223
orf8 protein                    300
spike glycoprotein             2019
dtype: int64

Percentage of the distribution of labelled records:
class
envelope protein                8.55
membrane glycoprotein           6.94
nucleocapsid phosphoprotein    18.62
orf10 protein                   1.39
orf3 protein                   11.95
orf6 protein                    2.99
orf7a protein                   6.82
orf7b protein                   3.75
orf8 protein                    5.04
spike glycoprotein             33.93
dtype: float64


------------
**Balance the data**

1. Downsample high frequency classes

In [24]:
for protein_name in ['nucleocapsid phosphoprotein', 'spike glycoprotein', 'orf3 protein']:
  # Separate majority and minority classes
  df_majority = corona_seq.loc[corona_seq['class'] == protein_name]
  df_minority = corona_seq.loc[corona_seq['class'] != protein_name]

  # Downsample majority class
  df_majority_downsampled = resample(df_majority, 
                                 replace=False,    # sample without replacement
                                 n_samples=500,    # to match minority class
                                 random_state=123) # reproducible results
  # Concatenate both dataframes again
  corona_seq =  pd.concat([df_majority_downsampled, df_minority])

2. Upsample low frequency classes  

In [25]:
for protein_name in ['membrane glycoprotein', 'orf8 protein', 'orf7a protein', 'orf7b protein', 'orf6 protein','orf10 protein']:
  # Separate majority and minority classes
  df_majority = corona_seq.loc[corona_seq['class'] != protein_name]
  df_minority = corona_seq.loc[corona_seq['class'] == protein_name]

  # Downsample majority class
  df_minority_upsampled = resample(df_minority, 
                                 replace=True,    # sample without replacement
                                 n_samples=500,     # to match minority class
                                 random_state=123) # reproducible results
  # Concatenate both dataframes again
  corona_seq =  pd.concat([df_minority_upsampled, df_majority])

  corona_seq = corona_seq.sample(frac=1,random_state=4).reset_index(drop=True)

In [26]:
class_distribution()

Counts of proteins:
 class
envelope protein               509
membrane glycoprotein          500
nucleocapsid phosphoprotein    500
orf10 protein                  500
orf3 protein                   500
orf6 protein                   500
orf7a protein                  500
orf7b protein                  500
orf8 protein                   500
spike glycoprotein             500
dtype: int64

Percentage of the distribution of labelled records:
class
envelope protein               10.16
membrane glycoprotein           9.98
nucleocapsid phosphoprotein     9.98
orf10 protein                   9.98
orf3 protein                    9.98
orf6 protein                    9.98
orf7a protein                   9.98
orf7b protein                   9.98
orf8 protein                    9.98
spike glycoprotein              9.98
dtype: float64


------------
Separate labels, encode them

In [27]:
y_corona = corona_seq.iloc[:, 1].values # y_corona for corona_seq
#Encode labels
labelencoder = LabelEncoder()
y_corona = labelencoder.fit_transform(y_corona)   

In [28]:
le_dict = dict(zip(labelencoder.transform(labelencoder.classes_), labelencoder.classes_))
#Check dictionary
le_dict

{0: 'envelope protein',
 1: 'membrane glycoprotein',
 2: 'nucleocapsid phosphoprotein',
 3: 'orf10 protein',
 4: 'orf3 protein',
 5: 'orf6 protein',
 6: 'orf7a protein',
 7: 'orf7b protein',
 8: 'orf8 protein',
 9: 'spike glycoprotein'}

------
Join words with an empty space

In [29]:
corona_seq['joined_words'] = list(corona_seq['words'])
for item in range(len(corona_seq['joined_words'])):
   corona_seq['joined_words'][item] = ' '.join([word for word in corona_seq['joined_words'][item] if word not in ['nnnnnn']])

In [30]:
corona_seq['joined_words'].head(5)

0    atggat tggatt ggattt gatttg atttgt tttgtt ttgt...
1    atrggc trggct rggcta ggctat gctata ctatat tata...
2    atggat tggatt ggattt gatttg atttgt tttgtt ttgt...
3    atgttt tgtttg gtttgt tttgtt ttgttt tgtttt gttt...
4    gttcta ttctaa tctaaa ctaaat taaatc aaatca aatc...
Name: joined_words, dtype: object

------
Count vectorizer - similar to bag-of-words - only thing is we do not have distinct vocabulary.

In [31]:
cv = CountVectorizer(ngram_range=(int(get_config_var('Jupyter Experiment', 'min_ngram')),int(get_config_var('Jupyter Experiment', 'max_ngram'))))

----
Split the data for training and testing, check their sizes and class distribution

In [32]:
X_train, X_test, y_train, y_test = get_cv_train_test_data()



Data shape - Train, Test: (3756, 11984) , (1253, 11984)
Label shape - Train, Test: (3756,) , (1253,)


Test Label values and counts:
 [(0, 128), (1, 125), (2, 125), (3, 125), (4, 125), (5, 125), (6, 125), (7, 125), (8, 125), (9, 125)]


------
If dimension reduction method is PCA, apply PCA get 9 components, scale the data with min max scaler. Later train the model of type 
Multi-Nomial Naive Bayes Algorithm and test. Print the results.

If dimension reduction method is Map-Elites, reduce dimensions with Map-Elites evalutionary approach and print the solutions.

In [33]:
if get_config_var('Jupyter Experiment', 'dim_reduction_type') == 'PCA':
    xtrn, xtst = dimension_reduction_pca(X_train, X_test,9)
    start = time.time()
    clf = clfMultinomialNB(X_train, X_test, y_train, y_test)


Proportion of variance = 
 [0.62774528 0.10387606 0.0621773  0.0510527  0.04284345 0.02457202
 0.01868454 0.00924098 0.00665491]

Total of variance covered  = 
 95.0 %


Algorithm Name: MultinomialNB
___________________________________________________________________


training model.........

testing.........

___________________________________________________________________

Experiment Information: Feature Reduction  with  PCA 

___________________________________________________________________

Pre-processing - kmer size: 6
Pre-processing - min_ngram and max_ngram: 1 , 1
dim_reduction_type: PCA

___________________________________________________________________
Results as below:
___________________________________________________________________


Confusion matrix:

Predicted    0    1    2    3    4    5    6    7    8    9
Actual                                                     
0          128    0    0    0    0    0    0    0    0    0
1            0  125    0    0    0 

-----

##### Genetic Algorithm

In [131]:
no_generations = int(get_config_var('GA Setup', 'no_generations'))
no_individuals = int(get_config_var('GA Setup', 'no_individuals'))
crossover_rate = float(get_config_var('GA Setup', 'crossover_rate'))
mutation_rate = float(get_config_var('GA Setup', 'mutation_rate'))

In [132]:
def generateES(ind_cls, strg_cls, size):
    #print('generateES1',ind_cls, strg_cls, size)
    ind = ind_cls(random.randint(0,X_train.shape[1]-1) for _ in range(size))
    #print('generateES2',ind)
    ind.strategy = strg_cls(random.randint(0,X_train.shape[1]-1) for _ in range(size))
    #print('generateES3',ind.strategy)
    return ind

def selElitistAndTournament(individuals, k, frac_elitist, tournsize):
    return tools.selBest(individuals, int(k*frac_elitist)) + tools.selTournament(individuals, int(k*(1-frac_elitist)), tournsize=tournsize)

def get_data_using_indexes(binary_featureindices):
    X_trn = X_train[:,binary_featureindices]
    X_tst = X_test[:,binary_featureindices]
    return X_trn, X_tst

def eval_model_function(individual):
    #print(f'GA-Individual: {individual}')
    X_trn, X_tst = get_data_using_indexes(individual)
    
    clf = MultinomialNB(alpha=0.1)
    clf.fit(X_trn, y_train)
    y_pred = predict(clf,X_tst)
    accuracy = accuracy_score(y_test, y_pred)
    fitness = accuracy
    
    #print(f'GA-fitness from Eval function:{fitness}')
    return fitness,

In [139]:
creator.create('FitnessMax', base.Fitness, weights=(1.0,))
creator.create('Individual', list, typecode="I", fitness=creator.FitnessMax, strategy=None)
creator.create("Strategy", list, typecode="I")

individual_size = (int(get_config_var('GA Setup', 'featureset_size')))

toolbox = base.Toolbox()
toolbox = base.Toolbox()

# generation functions
toolbox.register("individual", generateES, creator.Individual, creator.Strategy, individual_size)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

toolbox.register("mate", tools.cxOnePoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=0.05)

toolbox.register("select", selElitistAndTournament, frac_elitist=0.1 , tournsize=3)

toolbox.register("evaluate", eval_model_function)

# initialize parameters
pop = toolbox.population(n=no_individuals)
hof = tools.HallOfFame(no_individuals * no_generations)
stats = tools.Statistics(lambda ind: ind.fitness.values)
stats.register("avg", np.mean)
stats.register("min", np.min)
stats.register("max", np.max)
stats.register("std", np.std)

# genetic algorithm
pop, logbook = algorithms.eaSimple(pop, toolbox, cxpb=crossover_rate, mutpb=mutation_rate,ngen=no_generations, stats=stats, halloffame=hof,verbose=False)

#print('GA-Logbook:', logbook)
#print('GA-Best possible candidates:')

#cnt=0
#for top in hof.items:
#    cnt = cnt+1
#    print(f'GA--Generation: {cnt}, Top featureset: {top}')

print("Original dimension",X_train.shape)
print("Reduced dimension",X_trn.shape)

#Run with lowest scoring featureset
X_trn, X_tst = get_data_using_indexes(hof.items[100])
clf = MultinomialNB(alpha=0.1)
clf.fit(X_trn, y_train)
y_pred = predict(clf,X_tst)
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy with with lowest scoring featureset', accuracy * 100)

#Run with top scoring featureset
X_trn, X_tst = get_data_using_indexes(hof.items[0])
clf = MultinomialNB(alpha=0.1)
clf.fit(X_trn, y_train)
y_pred = predict(clf,X_tst)
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy with with top scoring (optimized) featureset', accuracy * 100)

/ichec/home/users/i18205615/.conda/envs/18205615/lib/python3.7/site-packages/deap/creator.py:141: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
/ichec/home/users/i18205615/.conda/envs/18205615/lib/python3.7/site-packages/deap/creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
/ichec/home/users/i18205615/.conda/envs/18205615/lib/python3.7/site-packages/deap/creator.py:141: RuntimeWarning: A class named 'Strategy' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


Original dimension (3756, 11984)
Reduced dimension (3756, 10)
Accuracy with with lowest scoring featureset 84.19792498004789
Accuracy with with top scoring (optimized) featureset 94.33359936153232


----------------------
##### Map-Elites